In [1]:
#Import packages
import os
import glob
import json
#from tqdm import tqdm

import nilearn
import nibabel as nib
from nilearn import image as nimg
from nilearn import plotting as nplot
from nilearn.glm.first_level import FirstLevelModel, make_first_level_design_matrix, run_glm
from nilearn.glm.second_level import make_second_level_design_matrix, SecondLevelModel
from nilearn.glm import fdr_threshold,threshold_stats_img
from nilearn.glm.contrasts import compute_contrast


from bids.layout import BIDSLayout, parse_file_entities

# import cortex
# from cortex import fmriprep

from nipype.interfaces.workbench.base import WBCommand
from nipype.algorithms import modelgen
from nipype.interfaces.base import Bunch

import scipy.stats as stats

import hcp_utils as hcp

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

%matplotlib inline

pixdim[1,2,3] should be non-zero; setting 0 dims to 1


In [2]:
#effect_size_df = pd.read_pickle('ACOMPCOR_sub_level_effect_size_df.pkl')
effect_size_df = pd.read_pickle('050923_effect_size_variance_df.pkl')
task_list = [tj.split('task-')[1].split('_bold')[0] for tj in glob.glob('../../task*json')]
tasks_included = ['pataka', 'emosent', 'vowel', 'nwr']
effect_size_df = effect_size_df[[c for c in effect_size_df.columns if any(t in c for t in tasks_included)]]

#TEMPORARILY DROPPING 955 BECAUSE THEY APPEAR TWICE IN THE DEMOGRAPHIC INFO
effect_size_df = effect_size_df.drop([c for c in effect_size_df if '955' in c], axis=1)

In [3]:
effect_size_df

,sub-voice884_ses-1_task-nwr_run-1,sub-voice884_ses-1_task-nwr_run-3,sub-voice884_ses-1_task-pataka_run-2,sub-voice884_ses-1_task-nwr_run-2,sub-voice884_ses-1_task-emosent_run-1,sub-voice884_ses-1_task-vowel_run-2,sub-voice884_ses-1_task-emosent_run-2,sub-voice884_ses-1_task-pataka_run-1,sub-voice884_ses-1_task-vowel_run-1,sub-voice857_ses-1_task-vowel_run-2,...,sub-voice872_ses-1_task-pataka_run-1,sub-voice872_ses-1_task-nwr_run-1,sub-voice872_ses-1_task-pataka_run-2,sub-voice979_ses-1_task-nwr_run-1,sub-voice979_ses-1_task-nwr_run-2,sub-voice979_ses-1_task-vowel_run-2,sub-voice979_ses-1_task-emosent_run-1,sub-voice979_ses-1_task-emosent_run-2,sub-voice979_ses-1_task-nwr_run-3,sub-voice979_ses-1_task-vowel_run-1
0,973.370547,1396.594872,2.308232,926.751624,134.826566,222.669111,593.235899,1.494750,126.937353,243.011514,...,0.962419,183.432843,1.802220,608.112024,1254.979886,467.451463,296.095399,665.433714,430.539121,366.176231
1,303.581573,1367.495897,3.199451,1070.681821,100.139881,243.409789,793.742717,2.568314,215.831074,92.204864,...,0.937923,182.936388,2.722800,161.339429,238.205189,70.271065,65.230744,106.207985,103.151166,58.457107
2,790.903031,617.729096,2.681006,1119.199486,212.874215,220.135285,683.347564,2.509075,301.521898,135.475896,...,3.608721,204.990356,11.098431,311.844201,412.120880,54.432855,71.796447,86.134674,380.367769,53.340547
3,361.603160,367.965184,1.726341,767.481983,153.755710,400.311737,215.501860,1.206870,317.476127,209.194838,...,2.077551,320.804834,6.040626,101.257787,93.871632,56.913496,36.784579,74.525071,71.605617,25.174085
4,805.746274,1012.474850,2.780564,1763.617238,1432.086282,412.100702,12368.824940,2.283997,2095.618229,191.382586,...,3.601401,311.083687,13.841866,383.927301,321.546771,209.617390,90.753943,284.898719,269.611710,126.082893
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
91277,404.455443,1001.340493,1.576172,1022.785982,187.531540,355.126636,685.885054,1.971069,350.414588,73.806170,...,1.497328,149.576540,3.243397,222.632454,204.625419,127.437232,91.779035,132.309473,99.088114,109.784042
91278,2745.250855,3831.554028,1.920902,2303.540017,980.320470,492.177765,1030.422163,2.454692,684.116846,182.924832,...,2.364806,156.623141,3.481724,277.332755,553.997358,180.567609,151.587630,261.349934,474.951289,226.539288
91279,1524.219403,2358.044496,1.314443,1821.349259,654.622732,407.799847,861.798589,1.712294,434.962179,111.198326,...,2.017105,143.750400,2.886457,225.078490,453.937369,202.446035,161.467180,149.153573,245.330573,154.713301
91280,445.631555,1194.172672,1.372434,1234.001390,285.217135,312.347169,552.729317,1.699838,258.658148,62.772715,...,1.365366,168.969446,2.553803,218.210399,200.891949,143.491482,101.625707,110.374684,92.023010,109.738038


In [6]:
#root = '../../derivatives/speech_v_silence_first_level_fxs_surf/'
root = '../../derivatives/speech_v_silence_first_level_fx_var_surf/'
for c in  effect_size_df.columns:
    surf_file = generate_zscore_cifti_object(effect_size_df[c].to_numpy(),root,c)

In [4]:
def generate_zscore_cifti_object(thresholded_pool,root,base,sample_cifti_file='../../derivatives/fmriprep/sub-voice857/ses-1/smoothed/sub-voice857_ses-1_task-pataka_run-1_space-fsLR_den-91k_bold.dtseries.nii'):
    #takes 91k np array of parcels and generates cifti object
    
    sample_cifti = nimg.load_img(sample_cifti_file)
    time_axis, brain_model_axis = [sample_cifti.header.get_axis(i) for i in range(sample_cifti.ndim)]
    z = np.atleast_2d(thresholded_pool)
    scalar_axis = nib.cifti2.ScalarAxis(['effects_size'])  # Takes a list of names, one per row
    new_header = nib.Cifti2Header.from_axes([scalar_axis, brain_model_axis])
    #z = np.reshape(z, (-1, z.shape[0]))
    img = nib.Cifti2Image(z, new_header)
    
#     name = f'{task}_thresholded_pool.dtseries.nii'
    name = f'{root}{base}_fsLR_91k_speechvssilence_effect_size_variance.dscalar.nii'
    img.to_filename(name)
    return name